In [ ]:
!pip install 'shimmy>=2.0'
!pip install stable_baselines3

In [ ]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from scipy.sparse import csr_matrix, issparse
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import cosine
from stable_baselines3 import SAC


In [ ]:
# Power method for warm-start initialization
def power_method(A, num_iterations=10):
    x = np.random.randn(A.shape[0])
    x /= np.linalg.norm(x)
    for _ in range(num_iterations):
        x = A.dot(x) if issparse(A) else A @ x
        x /= np.linalg.norm(x) + 1e-8
    return x

# Define RL Environment for Eigenvector Search
class EigenvectorEnv(gym.Env):
    def __init__(self, A, target_eigenvalue):
        super(EigenvectorEnv, self).__init__()
        self.A = A
        self.target_eigenvalue = target_eigenvalue
        self.dim = A.shape[0]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.dim,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.dim,), dtype=np.float32)
        self.state = None
        # Compute Frobenius norm for normalization
        self.A_norm = np.sqrt(np.sum(A.data**2)) if issparse(A) else np.linalg.norm(A, 'fro')

    def reset(self):
        # Warm-start with power method
        self.state = power_method(self.A)
        return self.state

    def step(self, action):
        action = np.array(action)
        action /= np.linalg.norm(action) + 1e-8
        self.state = action
        # Compute residual norm
        if issparse(self.A):
            residual = self.A.dot(action) - self.target_eigenvalue * action
        else:
            residual = self.A @ action - self.target_eigenvalue * action
        residual_norm = np.linalg.norm(residual)
        # Logarithmic scaling of residual norm
        residual_reward = -np.log1p(residual_norm / (self.A_norm + 1e-8))
        # Compute Rayleigh quotient deviation
        rayleigh = (action @ (self.A.dot(action) if issparse(self.A) else self.A @ action)) / (np.dot(action, action) + 1e-8)
        rayleigh_deviation = np.abs(rayleigh - self.target_eigenvalue) / (np.abs(self.target_eigenvalue) + 1e-8)
        # Combined reward with increased Rayleigh weight
        reward = residual_reward - 0.5 * rayleigh_deviation
        done = False
        return self.state, reward, done, {}

# Function to compute cosine distance
def cosine_distance(v1, v2):
    v1 = v1 / (np.linalg.norm(v1) + 1e-8)
    v2 = v2 / (np.linalg.norm(v2) + 1e-8)
    cos_sim = np.abs(np.dot(v1, v2))
    cos_sim = min(cos_sim, 1.0)  # Handle numerical errors
    return np.arccos(cos_sim) / np.pi  # Normalized to [0, 1]

# Function to find dominant eigenvalue and compute eigenvector using RL
def compute_dominant_eigenvector(A, timesteps=200000, eval_steps=1000):
    # Ensure matrix is symmetric
    if issparse(A):
        A = (A + A.T) / 2
    else:
        A = (A + A.T) / 2

    # Compute dominant eigenvalue and eigenvector using scipy
    eigvals, eigvecs = eigsh(A, k=1, which='LA') if issparse(A) else np.linalg.eigh(A)
    dominant_eigenvalue = eigvals[-1] if not issparse(A) else eigvals[0]
    true_eigenvector = eigvecs[:, -1] if not issparse(A) else eigvecs[:, 0]
    print("Dominant eigenvalue (scipy):", dominant_eigenvalue)

    # Initialize environment and SAC model
    env = EigenvectorEnv(A, dominant_eigenvalue)
    policy_kwargs = dict(net_arch=[512, 256, 128])
    model = SAC(
        'MlpPolicy',
        env,
        verbose=1,
        learning_rate=0.0001,
        buffer_size=1000000,
        ent_coef='auto',
        policy_kwargs=policy_kwargs,
        device='cpu'  # Avoid GPU issues with MLP policy
    )
    model.learn(total_timesteps=timesteps)

    # Evaluate to find best eigenvector
    obs = env.reset()
    best_reward = -np.inf
    best_vec = None

    for _ in range(eval_steps):
        action, _ = model.predict(obs, deterministic=True)  # Use deterministic policy for evaluation
        obs, reward, _, _ = env.step(action)
        if reward > best_reward:
            best_reward = reward
            best_vec = obs

    print("Approximated eigenvector (RL):", best_vec)
    # Compute residual norm for verification
    residual_norm = np.linalg.norm(A @ best_vec - dominant_eigenvalue * best_vec) if not issparse(A) else np.linalg.norm(A.dot(best_vec) - dominant_eigenvalue * best_vec)
    print("Residual norm ||Ax - λx||:", residual_norm)
    # Compute cosine distance for accuracy
    cos_dist = cosine_distance(best_vec, true_eigenvector)
    print("Cosine distance to true eigenvector:", cos_dist)

    return dominant_eigenvalue, best_vec, cos_dist



# Test on Large sparse matrix

In [ ]:
# Test with large sparse symmetric matrix
from scipy.sparse import random as sparse_random
np.random.seed(42)
size = 1000
A_sparse = sparse_random(size, size, density=0.01, format='csr')
print("\nTesting with sparse matrix:")
eigenvalue, eigenvector, cos_dist_sparse = compute_dominant_eigenvector(A_sparse)

# Test on dense matrix

In [ ]:
# Test with dense symmetric matrix
np.random.seed(42)
size = 5
A_dense = np.random.randn(size, size)
A_dense = (A_dense + A_dense.T) / 2
print("\nTesting with dense matrix:")
eigenvalue, eigenvector, cos_dist_dense = compute_dominant_eigenvector(A_dense)

